### Домашнее задание к лекции Улучшение качества модели

Взять boston house-prices datase (sklearn.datasets.load_boston). Возмите 7 любых регрессоров (попробовать разные алгоритмы, поподбирать параметры, вывести итоговое качество).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.datasets import load_boston

In [32]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(load_boston()['filename'], skiprows=1)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


Пропусков в даных нет, обработка отсутствующих значений не нужна

In [4]:
# разбиваем датасет на целевую переменную и признаки
X, Y = df.iloc[:, :-1], df['MEDV']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
# отделяем валидационную выборку
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2)

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
# проведем нормализацию данных, чтобы привести признаки к единому масштабу
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)

Выберем модели для регрессионного анализа

In [10]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor

Заполним сетки для подбора параметров каждой модели

In [28]:
params_lr = {'positive': [True, False]}
params_rd = {'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'], 'alpha': np.linspace(0.1, 5, 20)}
params_ls = {'alpha': np.linspace(0.1, 5, 20), 'selection': ['cyclic', 'random']}
params_rfr = {'n_estimators': [10, 30, 50, 70, 100], 
              'max_depth':np.arange(1,8), 
              'min_samples_split': np.arange(2,10), 
              'min_samples_leaf':np.arange(1,5)}
params_dtr = {'splitter':['best','random'],
              'max_depth':np.arange(1,8), 
              'min_samples_split':np.arange(2,10),
              'min_samples_leaf':np.arange(1,5)}
params_svr = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma': ['scale', 'auto'], 'C':[0.1,0.3,0.5,0.7,1]}
params_sgd = {'penalty': ['l1', 'l2', 'elasticnet'], 
              'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
              'alpha': [0.0001, 0.001, 0.01]}

In [29]:
# список моделей
models = [
    {'name': 'Lr', 'model': LinearRegression(), 'params': params_lr},
    {'name': 'Rd', 'model': Ridge(), 'params': params_rd},
    {'name': 'Ls', 'model': Lasso(), 'params': params_ls},
    {'name': 'Svr', 'model': SVR(), 'params': params_svr},
    {'name': 'Rfr', 'model': RandomForestRegressor(), 'params': params_rfr},
    {'name': 'Dtr', 'model': DecisionTreeRegressor(), 'params': params_dtr},
    {'name': 'Sgd', 'model': SGDRegressor(), 'params': params_sgd}
]

In [15]:
from sklearn.model_selection import RandomizedSearchCV

In [33]:
res=[]
for v in  models:
    res.append((v['name'], RandomizedSearchCV(v['model'], v['params'], cv=10).fit(X_train, y_train)))

In [34]:
for r in res:
    print(r[0], r[1].best_score_, r[1].best_params_)

Lr 0.7273094423067769 {'positive': False}
Rd 0.7292824488428481 {'solver': 'cholesky', 'alpha': 5.0}
Ls 0.6955937350872903 {'selection': 'random', 'alpha': 0.35789473684210527}
Svr 0.7168924243671102 {'kernel': 'linear', 'gamma': 'scale', 'C': 0.5}
Rfr 0.8323853341613507 {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_depth': 5}
Dtr 0.7706675390161289 {'splitter': 'best', 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 7}
Sgd 0.7301396865608737 {'penalty': 'l2', 'loss': 'squared_loss', 'alpha': 0.01}


Лучший результат показали модели Random forest regressor и decision tree regressor. Попробуем для них отдельно провести поиск лучших параметров по сетке рядом со значениями, которые были выбраны в результате работы RandomizedSearchCV 

In [35]:
from sklearn.model_selection import GridSearchCV

In [37]:
params_rfr_grid = {'n_estimators': [80, 90, 100, 110, 120], 
              'max_depth':np.arange(4,8), 
              'min_samples_split': np.arange(2,5), 
              'min_samples_leaf':np.arange(1,5)}
params_dtr_grid = {'splitter':['best'],
              'max_depth':np.arange(6,9), 
              'min_samples_split':np.arange(2,5),
              'min_samples_leaf':np.arange(3,6)}

In [45]:
grid_rfr = GridSearchCV(RandomForestRegressor(), params_rfr_grid, cv=10, scoring='r2')
grid_rfr.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': array([4, 5, 6, 7]),
                         'min_samples_leaf': array([1, 2, 3, 4]),
                         'min_samples_split': array([2, 3, 4]),
                         'n_estimators': [80, 90, 100, 110, 120]},
             scoring='r2')

In [43]:
grid_dtr = GridSearchCV(DecisionTreeRegressor(), params_dtr_grid, cv=10, scoring='r2')
grid_dtr.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': array([6, 7, 8]),
                         'min_samples_leaf': array([3, 4, 5]),
                         'min_samples_split': array([2, 3, 4]),
                         'splitter': ['best']},
             scoring='r2')

In [46]:
print(f'Random forest regressor: score {grid_rfr.best_score_}, params {grid_rfr.best_params_}')

Random forest regressor: score 0.8527073386810269, params {'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 90}


In [44]:
print(f'Decision tree regressor: score {grid_dtr.best_score_}, params {grid_dtr.best_params_}')

Decision tree regressor: score 0.7842042777370414, params {'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 3, 'splitter': 'best'}


С новыми гиперпараметрами результат у Random forest regressor улучшился еще на 2%, у Decision tree regressor на 1,4% 

Посмотрим на результаты, которые обученные модели покажут на отложенной валидационной выборке

In [49]:
# модели с параметрами, подобранными после randomizedSearch 
for r in res:
    print(r[0], r[1].best_estimator_.score(X_valid, y_valid))

Lr 0.618967079242239
Rd 0.6150785817012956
Ls 0.5590319018453553
Svr 0.5405526567587893
Rfr 0.7071333412499351
Dtr 0.6752171304214307
Sgd 0.6069973558691024


In [47]:
# Random forest regressor после GridSearch
grid_rfr.best_estimator_.score(X_valid, y_valid)

0.7529091666135501

In [48]:
# Decision tree regressor после GridSearch
grid_dtr.best_estimator_.score(X_valid, y_valid)

0.684590744028653

Видим, что случайный лес и на валидационной выборке показывает лучший результат из всех, а после дополнительного подбора параметров этот результат вырастает на 5%